In [1]:
import numpy as np
import pandas as pd

In [5]:
# Reading user file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|',
                    names=u_cols, encoding='latin-1')
n_users = users.shape[0]
print('Number of users:', n_users)

# Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()
print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])


Number of users: 943
Number of traing rates: 90570
Number of test rates: 9430


### Xây dựng item profiles

In [8]:
# Reading items file:
i_cols = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# , encoding='latin-1')
items = pd.read_csv('ml-100k/u.item', sep='|',
                    names=i_cols, encoding='latin-1')

n_items = items.shape[0]
print('Number of items:', n_items)


Number of items: 1682


Vì ta đang dựa trên thể loại của phim để xây dựng profile, ta sẽ chỉ quan tâm tới 19 giá trị nhị phân ở cuối mỗi hàng:

In [9]:
X0 = items.to_numpy()
X_train_counts = X0[:, -19:]

TF-IDF

In [10]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
X = transformer.fit_transform(X_train_counts.tolist()).toarray()

Mỗi hàng của X tương ứng 1 vector đặc trưng của 1 bộ phim

In [13]:
print(rate_train[:4, :])

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]


### Xây dựng user profiles
Tiếp theo, với mỗi user, chúng ta cần xây dựng những bộ phim nào mà user đó đã rated, và giá trị của các rating đó.

In [14]:
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:, 0]  # all users

    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1
    # but id in python starts from 0
    ids = np.where(y == user_id + 1)[0]
    item_ids = rate_matrix[ids, 1] - 1  # index starts from 0
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)


## Train
Bây giờ, ta có thể đi tìm các hệ số của Ridge Regression cho mỗi user:

In [15]:
from sklearn.linear_model import Ridge
# from sklearn import linear_model

d = X.shape[1]  # data dimension
W = np.zeros((d, n_users))
b = np.zeros(n_users)

for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    model = Ridge(alpha=0.01, fit_intercept=True)
    Xhat = X[ids, :]
    model.fit(Xhat, scores)
    W[:, n] = model.coef_
    b[n] = model.intercept_


Sau khi tính được các hệ số W và b, ratings cho mỗi items được dự đoán bằng cách tính:

In [16]:
# predicted scores
Yhat = X.dot(W) + b

In [22]:
# Dưới đây là một ví dụ với user có id là 100.
n = 100
np.set_printoptions(precision=2) # 2 digits after . 
ids, scores = get_items_rated_by_user(rate_test, n)
# Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [221 251 280 281 303 368 404 470 595 828]
True ratings     : [3 3 2 3 3 2 4 3 3 3]
Predicted ratings: [3.28 3.5  3.07 2.28 3.23 2.11 2.83 3.29 2.69 3.28]


## Evaluate
Để đánh giá mô hình tìm được, ta sẽ sử dụng Root Mean Squared Error (RMSE), tức căn bậc hai của trung bình cộng bình phương của lỗi. Lỗi được tính là hiệu của true rating và predicted rating:

In [23]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return np.sqrt(se/cnt)

print('RMSE for training: %.2f' %evaluate(Yhat, rate_train, W, b))
print('RMSE for test    : %.2f' %evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.91
RMSE for test    : 1.27


Như vậy, với tập training, sai số vào khoảng 0.9 sao; với tập test, sai số lớn hơn một chút, rơi vào khoảng 1.3. Kết quả này chưa thực sự tốt vì đã đơn giản hoá mô hình đi quá nhiều.